## Assignment

In this assignment, you'll continue working with the house prices data. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

* Load the **houseprices** data from Thinkful's database.
* Reimplement your model from the previous checkpoint.
* Try OLS, Lasso, Ridge, and ElasticNet regression using the same model specification. This time, you need to do **k-fold cross-validation** to choose the best hyperparameter values for your models. Scikit-learn has RidgeCV, LassoCV, and ElasticNetCV that you can utilize to do this. Which model is the best? Why?

This is not a graded checkpoint, but you should discuss your solution with your mentor. After you've submitted your work, take a moment to compare your solution to [this example solution](https://github.com/Thinkful-Ed/machine-learning-regression-problems/blob/master/notebooks/7.solution_overfitting_and_regularization.ipynb).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV

# Display preferences.
%matplotlib inline


import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [2]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df_houseprices = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [3]:
df_houseprices = pd.concat([df_houseprices,pd.get_dummies(df_houseprices.mszoning, prefix="mszoning", drop_first=True)], axis=1)
df_houseprices = pd.concat([df_houseprices,pd.get_dummies(df_houseprices.street, prefix="street", drop_first=True)], axis=1)
dummy_column_names = list(pd.get_dummies(df_houseprices.mszoning, prefix="mszoning", drop_first=True).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(df_houseprices.street, prefix="street", drop_first=True).columns)

In [21]:
df_houseprices['totalsf'] = df_houseprices['totalbsmtsf'] + df_houseprices['firstflrsf'] + df_houseprices['secondflrsf']
df_houseprices['int_over_sf'] = df_houseprices['totalsf'] * df_houseprices['overallqual']

# X is the feature set
X = df_houseprices[['overallqual',  'garagecars', 'grlivarea', 'garagearea', 'totalbsmtsf', 'totalsf', 'int_over_sf']  + dummy_column_names]
# Y is the target variable
Y = np.log1p(df_houseprices['saleprice'])

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}".format(X_test.shape[0]))

alphas = [np.power(10.0,p) for p in np.arange(-10,40,1)]

The number of observations in training set is 1168
The number of observations in test set is 292


In [22]:
# We fit an OLS model using sklearn
lrm = LinearRegression()
lrm.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in the training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in the training set is: 0.8322051418573924
-----Test set statistics-----
R-squared of the model in the test set is: 0.8246102692175237
Mean absolute error of the prediction is: 0.12576152172913144
Mean squared error of the prediction is: 0.029245474455144978
Root mean squared error of the prediction is: 0.171013082701719
Mean absolute percentage error of the prediction is: 1.0508733861606083


In [23]:
lasso_cv = LassoCV(alphas=alphas, cv=5)

lasso_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lasso_cv.predict(X_train)
y_preds_test = lasso_cv.predict(X_test)

print("Best alpha value is: {}".format(lasso_cv.alpha_))
print("R-squared of the model in training set is: {}".format(lasso_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lasso_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 0.0001
R-squared of the model in training set is: 0.83201241164664
-----Test set statistics-----
R-squared of the model in test set is: 0.8222901463386477
Mean absolute error of the prediction is: 0.12630116142429834
Mean squared error of the prediction is: 0.029632344849918092
Root mean squared error of the prediction is: 0.1721404799863126
Mean absolute percentage error of the prediction is: 1.0557557466929144


In [24]:
ridgeregr = RidgeCV(alphas=alphas, cv = 5) 
ridgeregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridgeregr.predict(X_train)
y_preds_test = ridgeregr.predict(X_test)

print("Best alpha value is: {}".format(ridgeregr.alpha_))
print("R-squared of the model on the training set is: {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(ridgeregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


Best alpha value is: 1.0
R-squared of the model on the training set is: 0.8317187172067856
-----Test set statistics-----
R-squared of the model on the test set is: 0.8199467107731441
Mean absolute error of the prediction is: 0.12678657761876208
Mean squared error of the prediction is: 0.0300231025337486
Root mean squared error of the prediction is: 0.17327175919274496
Mean absolute percentage error of the prediction is: 1.0601472586721667


In [25]:
elasticregr = ElasticNetCV(alphas=alphas, cv=5)
elasticregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticregr.predict(X_train)
y_preds_test = elasticregr.predict(X_test)

print("Best alpha value is: {}".format(elasticregr.alpha_))
print("R-squared of the model on the training set is: {}".format(elasticregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(elasticregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


Best alpha value is: 0.001
R-squared of the model on the training set is: 0.8301011345235119
-----Test set statistics-----
R-squared of the model on the test set is: 0.814623904644288
Mean absolute error of the prediction is: 0.1277381403927566
Mean squared error of the prediction is: 0.030910657295231247
Root mean squared error of the prediction is: 0.17581426931631927
Mean absolute percentage error of the prediction is: 1.0688224438616372


- With few exceptions, the majority of evaluation metrics suggest the best model is the initial OLS regression.
- R-squared is highest and the other eval metrics are lower compared to the rest.